In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import time

import torch
print(torch.cuda.is_available())  # True 表示可以用 GPU

cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
)
cfg.MODEL.DEVICE = "cuda"  # 指定使用 GPU

predictor = DefaultPredictor(cfg)

print("Detectron2 模型運行裝置:", predictor.model.device)

True
Detectron2 模型運行裝置: cuda:0


In [ ]:
# analyze_pitch_from_detectron2.py
# 👉 全自動流程：影片 → Detectron2 → 三動作偵測 → 特徵分析

import os
import json
import time
import shutil
import pandas as pd
from tqdm import tqdm
import torch
import warnings
warnings.filterwarnings("ignore")

from pose_utils_detectron2 import load_pose_sequence
from pose_detectron2 import run_detectron2

from detect_release import detect_release
from detect_landing import detect_landing
from detect_shoulder import detect_shoulder

from landing_features import detect_landing_features
from shoulder_features import extract_shoulder_features
from release_features import extract_release_features

# ✅ 單支影片轉特徵 json
def video_2_json(video_path, json_path, description, keypoint_dir, device):
    try:
        if os.path.exists(keypoint_dir):
            # 有舊資料，跳過後續動作
            print(f"{keypoint_dir} 已存在，跳過處理。")

        # 🧠 執行 detectron2 輸入影片 → 關節座標資料
        run_detectron2(video_path=video_path, output_dir=keypoint_dir, device=device)
        pose_sequence = load_pose_sequence(keypoint_dir)

        # 🎯 偵測三個關鍵偵
        release_frame = detect_release(pose_sequence)
        landing_frame = detect_landing(pose_sequence, release_frame)
        shoulder_frame = detect_shoulder(pose_sequence, release_frame)

        # 🔍 提取所有特徵
        all_features = {}

        landing_features = detect_landing_features(pose_sequence, landing_frame)
        for key, value in landing_features.items():
            all_features[f'landing_features_{key}'] = value

        shoulder_features = extract_shoulder_features(pose_sequence, shoulder_frame, landing_frame)
        for key, value in shoulder_features.items():
            all_features[f'shoulder_features_{key}'] = value

        release_features = extract_release_features(pose_sequence, release_frame)
        for key, value in release_features.items():
            all_features[f'release_features_{key}'] = value

        all_features['description'] = description

        # 💾 輸出 JSON
        assert ('data' in json_path) and ('features_and_labels' in json_path)
        os.makedirs(os.path.dirname(json_path), exist_ok=True)
        with open(json_path, 'w') as f:
            json.dump(all_features, f, indent=2)
        print('✅ 特徵已儲存至', json_path)

    except Exception as e:
        print('❌ 錯誤發生於影片:', video_path)
        print(e)

ROOT_DIR = 'data'

# ✅ 主程式
def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"⚙️ 使用裝置：{device}")

    for subdir in tqdm(os.listdir(ROOT_DIR)):
        subdir_path = os.path.join(ROOT_DIR, subdir)
        if not os.path.isdir(subdir_path):
            print('⚠️ 找不到子目錄')
            continue

        player_name = "_".join(subdir.split("_")[:3])
        csv_path = os.path.join(ROOT_DIR, subdir, f"{player_name}.csv")

        if not os.path.exists(csv_path):
            print(f"⚠️ 找不到對應的 CSV：{csv_path}")
            continue

        df = pd.read_csv(csv_path)

        for video_file in tqdm(os.listdir(subdir_path)):
            if not video_file.endswith('.mp4'):
                continue

            video_path = os.path.join(subdir_path, video_file)

            # ✅ 更穩健的檢查影片對應描述（忽略大小寫）
            row = df[df['Filename'].str.lower() == video_file.lower()]
            if row.empty:
                print(f"⚠️ 找不到影片對應的描述：{video_file}")
                continue

            description = row.iloc[0]['description']

            json_output_path = os.path.join(subdir_path, "features_and_labels", video_file.replace(".mp4", ".json"))
            keypoint_dir = os.path.join(subdir_path, "KEYPOINT_DIR", video_file.replace(".mp4", ""))

            video_2_json(video_path, json_output_path, description, keypoint_dir, device)

if __name__ == "__main__":
    main()


⚙️ 使用裝置：cuda


  0%|          | 0/202 [00:00<?, ?it/s]

🎞️ 處理影片：data/Gerrit_Cole_CH_videos_4S/pitch_0001.mp4，總影格：238，FPS: 59.61


  0%|          | 0/238 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1
✅ Detectron2 處理完畢，骨架資料與圖片已儲存。


...

👤 Frame 0: 設定目標人物 ID = 1
✅ Detectron2 處理完畢，骨架資料與圖片已儲存。



 17%|█▋        | 34/202 [1:45:26<7:01:38, 150.59s/it]

0 : 35.37127685546875 580.3424682617188 448.0974426269531
1 : 35.2177734375 579.9525756835938 447.8399963378906
2 : 35.225982666015625 579.9808349609375 447.7975158691406
3 : 35.23211669921875 579.986328125 447.81207275390625
4 : 35.421417236328125 580.1908569335938 447.88189697265625
5 : 35.364654541015625 580.7299194335938 447.68157958984375
6 : 34.405181884765625 581.0126953125 448.73211669921875
7 : 35.306060791015625 581.0112915039062 447.8414611816406
8 : 34.348968505859375 580.2958374023438 448.2049255371094
9 : 34.544036865234375 580.2045288085938 447.99102783203125
10 : 34.266204833984375 580.2822265625 447.8647766113281
11 : 35.303375244140625 580.0797729492188 447.9318542480469
12 : 34.566314697265625 579.9132080078125 448.448974609375
13 : 35.222198486328125 579.8472290039062 448.3788146972656
14 : 35.450531005859375 580.3486328125 448.0556640625
15 : 34.435943603515625 580.1591186523438 447.79779052734375
16 : 34.367584228515625 581.0863037109375 447.6591491699219
17 : 34.

  0%|          | 0/238 [00:00<?, ?it/s]

👤 Frame 0: 設定目標人物 ID = 1
